<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/k__Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/refs/heads/main/Data/penguins.csv")

In [3]:
df["island"].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [4]:
df['species'] = df.species.replace({'Adelie':0, 'Chinstrap':1, 'Gentoo':2})
df['island'] = df.island.replace({'Biscoe':0, 'Dream':1, 'Torgersen':2})
df['sex'] = df.sex.replace({'male':0, 'female':1, 'NA':2})

<ipython-input-4-3c10aee48c28>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['species'] = df.species.replace({'Adelie':0, 'Chinstrap':1, 'Gentoo':2})
<ipython-input-4-3c10aee48c28>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['island'] = df.island.replace({'Biscoe':0, 'Dream':1, 'Torgersen':2})
<ipython-input-4-3c10aee48c28>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in

In [5]:
train_set, test_set = train_test_split(df, test_size=0.2)

In [6]:
train_set = train_set[['species', 'island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
test_set = test_set[['species', 'island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]

In [25]:
X_train = train_set[['island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
y_train = train_set['species']
X_test = test_set[['island', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'sex', 'year']]
y_test = test_set['species']

In [26]:
num_attributes = X_train.select_dtypes( include = ['float64','int64']).columns
## This might be empty!? ##
cat_attributes = X_train.select_dtypes( include = ['object']).columns

In [27]:
trf = [ ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
         ('Scaler', StandardScaler())]),
         num_attributes), ('cat', OneHotEncoder( handle_unknown='ignore' ), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )

In [28]:
knn = KNeighborsClassifier(n_neighbors=5)

In [29]:
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', knn)])

In [30]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaler',
                                                                   StandardScaler())]),
                                                  Index(['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g', 'sex', 'year'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index([], dtype='object'))])),
                ('clf', KNeighborsClassifier())])

In [31]:
pipeline.score(X_train,y_train)

0.9963636363636363

In [32]:
pipeline.score(X_test,y_test)

0.9855072463768116

In [37]:
## Test knn on this unseen instance ##
unseen_instance = {'island':'Torgersen','bill_length_mm':40.5, 'bill_depth_mm':21,'flipper_length_mm':189, 'body_mass_g':3825, 'year':2008, 'sex':'male'}

In [38]:
unseen_instance = pd.DataFrame([unseen_instance])

In [39]:
feature_columns = pipeline.named_steps['pre'].transformers_[0][2]

In [40]:
unseen_instance = unseen_instance[feature_columns]

In [41]:
pipeline.predict(unseen_instance)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Torgersen'